# **``Landsat 8 imgs 500x500 m around poi all 11 bands``**

- dataset_v1

In [1]:
# ! pip install psycopg2-binary
# ! pip install postgres

# # # ! sudo apt install libpq-dev python3-dev 
# # ! sudo apt-get install libpq-dev python-dev -y --fix-missing


# ! sudo apt-get install postgresql-server-dev-all -y
# ! sudo apt-get install postgresql-common -y


# ! pip install datacube rioxarray odc-cloud
# ! pip install geopandas --upgrade

# this shit ain't working!

# Load metadata and dependencies

## Connect with earth engine

In [2]:
import ee
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
service_account = user_secrets.get_secret("google_service_account")
json_key_path = '/kaggle/input/gc-json-key/ticktickbloom-project-000e9745b830-json-key.json'
credentials = ee.ServiceAccountCredentials(service_account, json_key_path)

ee.Initialize(credentials)

# Import data laoding and manipulating libs

In [3]:
# ! pip install geemap -q

In [4]:
import datetime
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib
from joblib import Parallel, delayed

import geopy.distance as distance

import warnings
warnings.filterwarnings('ignore')

In [5]:
metadata = pd.read_csv('/kaggle/input/ticktickbloomdataset/metadata.csv')
sub_format = pd.read_csv('/kaggle/input/ticktickbloomdataset/submission_format.csv')
train_labels = pd.read_csv('/kaggle/input/ticktickbloomdataset/train_labels.csv')

In [6]:
metadata.date = pd.to_datetime(metadata.date)
metadata.sort_values(by='date', ascending=False)

,uid,latitude,longitude,date,split
6540,hfvr,36.796200,-121.782000,2021-12-29,test
6864,howu,36.708500,-121.749000,2021-12-29,test
12443,nsoi,36.736800,-121.734000,2021-12-29,test
14254,prfi,36.751800,-121.742000,2021-12-29,test
17559,thki,36.725400,-121.730000,2021-12-29,test
...,...,...,...,...,...
5317,fwbt,44.850500,-93.515700,2013-01-04,train
5566,gdxr,44.877646,-93.557842,2013-01-04,train
6144,guny,44.878889,-93.490833,2013-01-04,train
13644,paev,44.822478,-93.367962,2013-01-04,train


# Define config 

In [7]:
img_folder = 'landsat8_500m_v1'

img_metadata_cols = ['CLOUD_COVER', 'CLOUD_COVER_LAND', 'IMAGE_QUALITY_OLI', 'IMAGE_QUALITY_TIRS', 'DATE_ACQUIRED']



# Collect data

In [8]:
def get_bufferbox(lat, long, buffer=500, lat_first=False):
    distance_search = distance.distance(meters=buffer)  # 0.5 km around our point of intrest
    min_lat = distance_search.destination((lat, long), bearing=180)[0]
    min_long = distance_search.destination((lat, long), bearing=270)[1]
    max_lat = distance_search.destination((lat, long), bearing=0)[0]
    max_long = distance_search.destination((lat, long), bearing=90)[1]
    if lat_first == True:
        return (min_lat, min_long, max_lat, max_long)     # for ee.Geometry.Rectangle
    else:
        return (min_long, min_lat, max_long, max_lat)


def get_date_n_days_ago(cur_date: str, n_days: int =30) -> str:
    return str(cur_date - datetime.timedelta(days=n_days)).split(' ')[0]


def get_img_col_in_daterange(cur_date: str, lat_long: tuple, bands: list = ['B1', 'B2', 'B3', 'B4' ,'B5','B6','B7','B8','B9','B10','B11'])-> ee.ImageCollection:
    """returns img collection of 15 imgs from cur_date to cur_date -16 days contanning given (lat, long)"""
    
    lat, lng = lat_long
    img_col = ee.ImageCollection("LANDSAT/LC08/C02/T1"
    ).filterDate(
      start=get_date_n_days_ago(pd.to_datetime(cur_date)),
      opt_end=str(cur_date)
    ).filterBounds(
        ee.Geometry.Point(lng, lat)
    ).select(
        bands
    ).limit(15)

    return img_col  


def get_latest_img(img_col: ee.ImageCollection) -> ee.Image:
    """returns latest image and it's metadata from collection of images"""
    img_metadata = []
    # sort this image collection
    if int(img_col.size().getInfo()) >= 1:
        # if atleast 1 img in img_col sort by date and get first img
        my_img = img_col.limit(1, 'system:time_start', False).first()
        # add metadata of img to a df
        for col in img_metadata_cols:
            img_metadata.append(my_img.get(col).getInfo())
        return my_img, img_metadata
    else:
        # print("No image found")
        img_notfound.append(uid)
        return None, [np.nan]*len(img_metadata_cols)

  
# convert latest image into array of array bands 
def get_arr_col_from_img(img: ee.Image, img_metadata: list, lat_long: tuple) -> np.array:
    if img is not None:
        bands = img.bandNames().getInfo()
        bbox = get_bufferbox(lat_long[0], lat_long[1], lat_first=False)
        bbox = ee.Geometry.Rectangle(bbox)
        band_arrs = img.sampleRectangle(bbox)
        img_band_col = []
        for band in bands:
            band_arr = np.array(band_arrs.get(band).getInfo())
            if band_arr is not None:
                img_band_col.append(band_arr)
            else:
                img_band_col.append(None)  
                return None 
    return np.array(img_band_col + img_metadata)  # return both img_vals and img_metadata in single write to optimize.


# save the band_arr 
def save_bands(band_col:np.array, uid: str) -> None:
    
    if os.path.exists(f'{img_folder}') and os.path.isdir(f'{img_folder}'):
        joblib.dump(filename=f'{img_folder}/{uid}.npy', value=band_col)
    else:
        os.makedirs(f'{img_folder}')
        joblib.dump(filename=f'{img_folder}/{uid}.npy', value=band_col)


def collect_data(uid: str, lat_long: tuple, cur_date:str)-> list:

    img_col = get_img_col_in_daterange(cur_date = cur_date, lat_long=lat_long)
    my_img, img_metadata = get_latest_img(img_col)
    band_col = get_arr_col_from_img(my_img, img_metadata, lat_long=lat_long)
    save_bands(band_col, uid) 
#     # return band_col and cache them like 100 or so and then write to disk to optmize
    
#     # increament func call counter
#     cc+=1
    
    return band_col


In [9]:
def collect_data_wrapper(data: pd.Series, n: int):
    cur_date = str(data.loc[n, 'date']).split(' ')[0]  # only date needed
    uid = data.loc[n, 'uid']
    lat_long = tuple(data.loc[n][['latitude', 'longitude']])
    
    try:
        band_col = collect_data(cur_date=cur_date, uid=uid, lat_long=lat_long)
#         band_col_cache.append(band_col)
#         if n % 4 == 0:
#             for band_col in band_col_cache:
#                 save_bands(band_col, uid)
    except Exception as e:
        err_imgs.append(uid)
        errors.append(e)


In [10]:
%%time

# data = metadata.sample(30).reset_index(drop=True)
data = metadata

img_metadata_df = pd.DataFrame(columns=img_metadata_cols) # df to hold gathered imgs' metadata
err_imgs = []  # list of uids where img failed!
img_notfound = []
errors = []


# #  loop through whole dataset and collect imgs : 
# for n in tqdm(range(data.shape[0]), desc='Collecting Imgs: ', colour='green'):
#     collect_data_wrapper(data, n)

CPU times: user 1.99 ms, sys: 0 ns, total: 1.99 ms
Wall time: 1.99 ms


In [11]:
%%time

# parrllelize shit
Parallel(n_jobs=-1,backend='threading')(
    delayed(collect_data_wrapper)
    (data, n)
    for n in tqdm(range(data.shape[0]), desc='Collecting Imgs: ', colour='green')
)


CPU times: user 30min, sys: 1min 26s, total: 31min 27s
Wall time: 6h 21min 34s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [12]:
err_imgs

['aaia',
 'aaoy',
 'aauy',
 'abcx',
 'abdc',
 'abjx',
 'abvy',
 'abzb',
 'aclb',
 'acmt',
 'aczx',
 'adjx',
 'afhv',
 'afjf',
 'aflh',
 'agxe',
 'ahfv',
 'ahvc',
 'ahwi',
 'ahya',
 'aicl',
 'aigc',
 'ajuj',
 'ajvk',
 'ajxf',
 'akcy',
 'akvu',
 'alfh',
 'algu',
 'alhz',
 'alxv',
 'anlo',
 'aogc',
 'apph',
 'apqc',
 'aqef',
 'aqjx',
 'aqtk',
 'arnp',
 'arwv',
 'ascv',
 'ascz',
 'ashi',
 'ashp',
 'asrm',
 'asxp',
 'asyc',
 'atnl',
 'atog',
 'avfl',
 'awog',
 'axqb',
 'aydg',
 'ayfy',
 'aytj',
 'ayze',
 'azdp',
 'azql',
 'azvh',
 'baav',
 'baos',
 'bapo',
 'bbwo',
 'bcnm',
 'bcym',
 'begh',
 'beom',
 'beyu',
 'bfes',
 'bfgd',
 'bfip',
 'bfzl',
 'bgao',
 'bgci',
 'bgwz',
 'bhdp',
 'bhfm',
 'bhml',
 'bien',
 'biic',
 'biit',
 'bils',
 'bivo',
 'bjij',
 'bjpu',
 'bjxz',
 'bkud',
 'bmdf',
 'bmqu',
 'bmso',
 'bmwj',
 'bnnx',
 'bnzn',
 'bphe',
 'bqrl',
 'bsku',
 'btbu',
 'btfg',
 'buld',
 'buux',
 'bvda',
 'bvyn',
 'bwad',
 'bwku',
 'byfh',
 'byfj',
 'byhg',
 'byor',
 'caqq',
 'catm',
 'cayd',
 

In [13]:
errors

[ee.ee_exception.EEException("Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B11' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip()."),
 NameError("name 'uid' is not defined"),
 NameError("name 'uid' is not defined"),
 NameError("name 'uid' is not defined"),
 ee.ee_exception.EEException("Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B10' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip()."),
 ee.ee_exception.EEException("Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B3' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in

In [14]:
img_notfound

[]

In [15]:
len(os.listdir(f'/kaggle/working/{img_folder}'))

22306

In [16]:
# save files
joblib.dump(err_imgs, 'missed_imgs.joblib')
joblib.dump(img_notfound, 'imgs_not_found_in_col.joblib')
# img_metadata_df.to_csv(f'{img_folder}/{img_folder}_metadata.csv', index=False)

['imgs_not_found_in_col.joblib']

In [17]:
#  issues - 
#  1. not saving files
#  2. not updating df
#  3. unable to debug too...